#### In this assignment, we wrangle the data from controvertial comments data set.
#### We will perfomr initial cleanup of the data and create new columns to work with
#### We will thenvectorize the comments using  CountVectorizer and TF-IDF models.
#### we will then calculate the jaccard distance beween two samples of the data set to show similairy
#### Using keywords deemed toward 'liberal', 'conservative', 'Positive1','Positive2', and 'negative', we will quantify them and show scatter plot and stats

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import collections
import matplotlib.pyplot as plt


In [ ]:
def jaccard(a, b):
    c = a.intersection(b)
    #print("this is intersection", c)
    
    d=a.union(b)
    #print("this is union", d)
    _a = collections.Counter(a)
    _b = collections.Counter(b)
    c = (_a - _b) + (_b - _a)
    #print("c", c)
    n = sum(c.values())
    #print(_a)
    #print(_b)
    #print("_a and _b")
    #print ("n", n)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [ ]:
file ='controversial-comments.jsonl'
# possible orient value: split, records, index, columns, and values)
# The following file is a subset of above file with the 1st 233537 lines.
dataframe = pd.read_json("controversial-comments_small.jsonl",orient="columns",lines=True)

In [ ]:
dataframe.shape

In [ ]:
# take a 1000 record sample of data with no duplicate records
df1 = dataframe.sample(n=1000, replace=True, random_state=10)
df2 = dataframe.sample(n=1000, replace=True, random_state=20)

In [ ]:
df1.head()

In [ ]:
df2.head()

In [ ]:
# convert to lower case
def ToLower(string: str) -> str:
    return string.lower()

In [ ]:
df1['lower_txt'] = [ToLower(string) for string in df1['txt']]
df2['lower_txt'] = [ToLower(string) for string in df2['txt']]
df1.head(5)

In [ ]:
from nltk import sent_tokenize

# Tokenize words
df1['Tokenized'] = [sent_tokenize(string) for string in df1['lower_txt']]
df2['Tokenized'] = [sent_tokenize(string) for string in df2['lower_txt']]
df1.head(5)

In [ ]:
df1['Tokenized']

In [ ]:
import re
#remove everything that has numbers in it
def removeInvalidWords(text): 
    return re.sub(r"\d+", '',text)

In [ ]:
df1['CleanText'] = [removeInvalidWords(string) for string in df1['lower_txt']]
df1.head(5)

In [ ]:
df2['CleanText'] = [removeInvalidWords(string) for string in df2['lower_txt']]
df2.head(5)

#### Use CountVectorizer to create list of words and feature matrix

In [ ]:
vectorizer = CountVectorizer()
X1 = vectorizer.fit_transform(df1['CleanText'])
list1 = vectorizer.get_feature_names()
print(list1[:30])

In [ ]:
print( X1.toarray())
X2.shape

In [ ]:
vectorizer = CountVectorizer()
X2 = vectorizer.fit_transform(df2['CleanText'])
list2 = vectorizer.get_feature_names()
print(list2[:30])

In [ ]:
print( X2.toarray())
X2.shape

#### jaccard distance show the two samples are 90%  similar

In [ ]:
words1 = set(list1)
words2 = set(list2)

z = jaccard(words1, words2)
print("this is percent similarity", z*100)

#### Use TfidVectorizer  to create list of words and feature matrix

In [ ]:
#TfidVectorizer Example
#Provide a list of all words in the corpus

from sklearn.feature_extraction.text import TfidfVectorizer
corpus = df1['CleanText']
vectorizer = TfidfVectorizer()
X1 = vectorizer.fit_transform(corpus)
list1 = vectorizer.get_feature_names()[:30]
print(list1)

print(X1.shape)


In [ ]:
#TfidVectorizer Example
#Provide a list of all words in the corpus

from sklearn.feature_extraction.text import TfidfVectorizer
corpus = df2['CleanText']
vectorizer = TfidfVectorizer()
X2 = vectorizer.fit_transform(corpus)
list2 = vectorizer.get_feature_names()[:30]
print(list2)

print(X2.shape)

In [ ]:
words1 = set(list1)
words2 = set(list2)

z = jaccard(words1, words2)
print("this is percent similarity", z*100)

#### TfidVectorizer Vectorization yield 100% similarity

## sentiment analyis

In [ ]:
# Take a 10,000 sample
df = dataframe.sample(n=10000, replace=True, random_state=2)

In [ ]:
df['lower_txt'] = [ToLower(string) for string in df['txt']]
df['CleanText'] = [removeInvalidWords(string) for string in df['lower_txt']]

In [ ]:
df['Liberal'] = df.CleanText.str.count('hillary') +  df.CleanText.str.count('clinton') +  df.CleanText.str.count('liberal') +  df.CleanText.str.count('wellware')
df['Conservative']= df.CleanText.str.count('wall') +  df.CleanText.str.count('trump') +  df.CleanText.str.count('activits') + df.CleanText.str.count('taxes') 

In [ ]:
df[['Liberal','Conservative']].head()

In [ ]:
NumberofLiberalComments = df['Liberal'][df.Liberal != 0].size
NumberofConservativeComments = df['Conservative'][df.Conservative != 0].size

In [ ]:
print(NumberofLiberalComments)
print(NumberofConservativeComments)

In [ ]:
plt.scatter(range(0,df['Liberal'].size),df['Liberal'],c='blue')
plt.scatter(range(0,df['Conservative'].size),df['Conservative'],c='red')
plt.title('Scatter Plot')
plt.xlabel('comment numbers')
plt.ylabel('liberal/Conservative')
plt.show()

#### positive/negatve sentiment

In [ ]:
df['positive1'] = df.CleanText.str.count('good')
df['positive2']= df.CleanText.str.count('special')
df['negative'] = df.CleanText.str.count('bad')
df['TotScore'] = df.positive1 + df.positive2 - df.negative

In [ ]:
plt.scatter(range(0,df['positive1'].size),df['positive1'],c='blue')
plt.scatter(range(0,df['positive2'].size),df['positive2'],c='green')
plt.scatter(range(0,df['negative'].size),df['positive2'],c='red')
plt.title('Scatter Plot')
plt.xlabel('comment numbers')
plt.ylabel('Positive/Negative')
plt.show()